In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import time
import pandas as pd

# Chrome WebDriver 옵션 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # 브라우저 GUI 비활성화
chrome_options.add_argument('--no-sandbox')  # 샌드박스 옵션 비활성화
chrome_options.add_argument('--disable-dev-shm-usage')  # 리소스 제한 방지
chrome_options.add_argument('lang=ko_KR')  # 브라우저 언어 설정

# 웹 크롤링 시작
browser = webdriver.Chrome(options=chrome_options)
browser.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')
browser.implicitly_wait(5)

columns = ['매장명', '위도', '경도', '매장타입', '주소']
all_starbucks_list = []  # 모든 매장 데이터를 저장할 리스트
save_loc = '/content/drive/MyDrive/Colab Notebooks/AI와데이터기초/personnal'

# 매장 정보 크롤링 함수
def crawl_starbucks_by_region(region_index):
    try:
        loc_search_button = browser.find_element(By.XPATH, '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/header[2]/h3/a')
        browser.execute_script("arguments[0].click();", loc_search_button)
        time.sleep(5)

        region_button = browser.find_element(By.XPATH, f'//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[{region_index}]/a')
        region_name = region_button.text
        print(f"{region_index}번째 지역: {region_name}")
        region_button.click()
        time.sleep(2)

        if region_index < 17:
            all_button = browser.find_element(By.XPATH, '//*[@id="mCSB_2_container"]/ul/li[1]/a')
            all_button.click()
            time.sleep(5)

        stores = browser.find_elements(By.CLASS_NAME, 'quickResultLstCon')
        print(f"{region_name} 매장 수: {len(stores)}")

        region_stores = []
        for store in stores:
            name = store.get_attribute('data-name')
            lat = store.get_attribute('data-lat')
            lng = store.get_attribute('data-long')
            store_type = "리저브" if name.endswith('R') else ("드라이브스루" if name.endswith('DT') else "일반")
            address = str(store.find_element(By.CLASS_NAME, 'result_details').get_attribute('outerHTML')).split('>')[1].split('<br')[0].strip()
            region_stores.append([name, lat, lng, store_type, address])

        df = pd.DataFrame(region_stores, columns=columns)
        df.to_excel(f'{save_loc}/starbucks_store_list_{region_name}.xlsx', index=False)
        print(f"{region_name} 저장 완료")

        return region_stores

    except Exception as e:
        print(f"오류 발생: {e}")
        return []

# 모든 지역 크롤링
for i in range(1, 18):
    print(f"----- {i}번째 지역 크롤링 시작 -----")
    region_stores = crawl_starbucks_by_region(i)
    all_starbucks_list.extend(region_stores)
    print(f"----- {i}번째 지역 크롤링 완료 -----")

# 전체 데이터 저장
all_starbucks_df = pd.DataFrame(all_starbucks_list, columns=columns)
all_starbucks_df.to_excel(f'{save_loc}/all_starbucks_store_list.xlsx', index=False)
print("크롤링 완료. 모든 데이터 저장.")
browser.quit()